## collecting samples from webcam

In [1]:
pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\pc-lenovo\anaconda\envs\tensorflow\lib\site-packages (20.1.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imgaug


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('C:/Users/PC-LENOVO/anaconda/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

def face_extractor(img): # so the image we work under gray scale not in rgb
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,6)
     # 1.3 is scale factor
     # 5 is minimum numbers of neighbours its value vary between 3 and 6 but the value at 6 or nearer to 6 give good result
     
    if faces is (): # if face is not present
        return None
    
    for (x,y,w,h) in faces:  # for loop is for cropping frontal faces to see ity
        cropped_face = img[y:y+h,x:x+w] # y is for rows and x is for columns
    
    return cropped_face
    
    
    
    
cap = cv2.VideoCapture(0) # 0 is our by default camera id
count = 0  # this variable is derive to take the number of photos

while True :
    ret, frame = cap.read()
     
    if face_extractor(frame) is not None:
         count+=1
         face = cv2.resize(face_extractor(frame),(200,200)) # we resize the image to 200,200
         face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY) # now convert that img to gray
         file_name_path = 'C:/Users/PC-LENOVO/Downloads/facial tutorial/faces/user'+str(count)+'.jpg' # now we save the face value so for that we have to define a path to aparticular file # user we define in path is the name of the samples we collected and count should be depicted in string format in jpg
         cv2.imwrite(file_name_path,face) # so it basically saving it for that we pass path and image
         cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2) # 50,50 is size it detect 0,255,0 define green color rbg 1 defines the scale 2 defines the font size
         cv2.imshow('Face Cropper',face) # it shows us current face samples
    else:
        print("Face not found")
        pass
    
    if cv2.waitKey(1)==13 or count==100:  #  now for closing it  13 is the asci code for enter to stop capturing photo or it will take 100 samples 
        break
        
        
cap.release() # now we have to release the camera
cv2.destroyAllWindows()
print("collecting samples complete!!!")
        
        

        
        
        

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
collecting samples complete!!!


# training that model by using those samples

In [5]:
import cv2
import numpy as np
from os import listdir # this helps in fetching data from current working directory
from os.path import isfile, join

data_path = 'C:/Users/PC-LENOVO/Downloads/facial tutorial/faces/' # so we place / at the end of working directory to fetch the data
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))] # so it basically join the corrent working directory to fetch the data stored in it

Training_Data, Labels = [],[] # so we have two empty list to insert training data and labels

for i,files in enumerate(onlyfiles): # enumerate provide the number of iteration according to the amount of data saved inside it
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE) # so it defines images we are fetching is in gray scale
    Training_Data.append(np.asarray(images,dtype=np.uint8)) # uint is unsigned integer
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

model = cv2.face.LBPHFaceRecognizer_create() # LINEAR BINARY PHASE HISTROGRAM RECOGNIZER so our classification model is built up

model.train(np.asarray(Training_Data),np.asarray(Labels))

print("Model Training is Completed")



Model Training is Completed


## testing the model

In [ ]:
face_classifier = cv2.CascadeClassifier('C:/Users/PC-LENOVO/anaconda/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # so it convert img to grayscale
    faces = face_classifier.detectMultiScale(gray,1.3,6)
    
    if faces is():# image is not present
        return img,[]

    for(x,y,w,h) in faces: # we scan each area of faces
        cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,0),2) # so it shows rectangle through x,y and x+w,y+h this shows how much increament should be done in the face and (0,255,0) shows the color of the rectangle 2 shows the thickness of the rectangle
        roi = img[y:y+h,x:x+w]# region of interest
        roi = cv2.resize(roi,(200,200))
    
    return img,roi

cap = cv2.VideoCapture(0)
while True:
    
    ret,frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        result = model.predict(face)
        
        if result[1] < 500: # if the first value in result is lesser than 500
            confidence = int(100*(1-(result[1])/300)) # we find the confidence value which describe how much percentage of faces will match
            display_string = str(confidence)+'% Confidence it is user' # it prints the integer value
           
        cv2.putText(image,display_string,(100,120),cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)# (250,120,255) is a color (100,120) is size 1 is scale 2 is font
        
        if confidence >75:
            cv2.putText(image,"Unlocked",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2) 
            cv2.imshow('Face Cropper', image)
        
        else:
            cv2.putText(image,"Locked",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2) 
            cv2.imshow('Face Cropper', image)
        
    except:
            cv2.putText(image,"Face Not Found",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2) 
            cv2.imshow('Face Cropper', image)
            pass # it heps in countinous running
        
        
    if cv2.waitKey(1)==13:
        break
        
cap.release()
cv2.destroyAllWindows()
    
    